In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install cvxpy

     |████████████████████████████████| 993kB 2.9MB/s eta 0:00:01     |█████████████▋                  | 419kB 2.9MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.6MB 645kB/s eta 0:00:01
     |████████████████████████████████| 153kB 8.7MB/s eta 0:00:01
     |████████████████████████████████| 215kB 6.2MB/s eta 0:00:01
  Created wheel for cvxpy: filename=cvxpy-1.1.0-cp36-cp36m-linux_x86_64.whl size=2498522 sha256=88c5314cb334db6eeb989258f918620b9697ce71505b97a939346daf3b6d7835
  Stored in directory: /home/nbuser/.cache/pip/wheels/6d/3e/58/c59291c242e379ac92b031c23a9356faa082a012ba620524b8
Successfully built cvxpy
  Created wheel for scs: filename=scs-2.1.2-cp36-cp36m-linux_x86_64.whl size=376074 sha256=792b46d601040340783cf1ec3710043bd8e9f10c4b4f977ee580a7700eb795a2
  Stored in directory: /home/nbuser/.cache/pip/wheels/df/d0/79/37ea880586da03c620ca9ecd5e

In [3]:
import cvxpy as cp

In [4]:
#First, need a function that picks elements from list L with probabilies probs
def pick_element_per_probs(L,probs):
    #make sure probs are normalized
    probs = [x / sum(probs) for x in probs]
    #pick a random number
    r = np.random.random()
    c = 0 #cumulative sum
    for i, prob in enumerate(probs):
        if r < prob + c:
            return L[i]
        c += prob

# Read in Data

In [5]:
post_cut = True

In [6]:
url = 'https://www.draftkings.com/lineup/getavailableplayerscsv?contestTypeId=86&draftGroupId=35810'
df = pd.read_csv(url)
#df = pd.read_csv('Data/DKSalaries_outlaw_20200513.csv')
df.head(14)

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,G,Sahith Theegala (14591201),Sahith Theegala,14591201,G,10400,Lone Tree Classic,Golf,0
1,G,Alistair Docherty (14591202),Alistair Docherty,14591202,G,10200,Lone Tree Classic,Golf,0
2,G,Brady Calkins (14591203),Brady Calkins,14591203,G,10000,Lone Tree Classic,Golf,0
3,G,Tyler Weworski (14591204),Tyler Weworski,14591204,G,9700,Lone Tree Classic,Golf,0
4,G,Nicolo Galletti (14591205),Nicolo Galletti,14591205,G,9400,Lone Tree Classic,Golf,0
5,G,Andrew Funk (14591206),Andrew Funk,14591206,G,9200,Lone Tree Classic,Golf,0
6,G,Neil Johnson (14591207),Neil Johnson,14591207,G,9000,Lone Tree Classic,Golf,0
7,G,Mark Baldwin (14591208),Mark Baldwin,14591208,G,8700,Lone Tree Classic,Golf,0
8,G,Samuel Saunders (14591209),Samuel Saunders,14591209,G,8400,Lone Tree Classic,Golf,0
9,G,Trevor Lampson (14591210),Trevor Lampson,14591210,G,8100,Lone Tree Classic,Golf,0


In [7]:
#df.to_csv('Data/DKSalaries_outlaw_20200512.csv',index=False)

In [8]:
#remove
to_remove = []
df = df[~df.Name.isin(to_remove)]
df.iloc[8:12]

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
8,G,Samuel Saunders (14591209),Samuel Saunders,14591209,G,8400,Lone Tree Classic,Golf,0
9,G,Trevor Lampson (14591210),Trevor Lampson,14591210,G,8100,Lone Tree Classic,Golf,0
10,G,Kolton Lapa (14591211),Kolton Lapa,14591211,G,7900,Lone Tree Classic,Golf,0
11,G,Daniel Hudson (14591212),Daniel Hudson,14591212,G,7700,Lone Tree Classic,Golf,0


In [9]:
df_made_cut = pd.read_csv('Data/outlaw_made_cut.csv')
df_made_cut.head(2)

,Name,Score
0,"Tyler Weworski Scottsdale, AZ",-16
1,"Sahith Theegala Chino Hills, CA",-15


In [10]:
df_made_cut.Name = df_made_cut.Name.str.split("  ",expand=True)[0]
df_made_cut.head()


,Name,Score
0,Tyler Weworski,-16
1,Sahith Theegala,-15
2,Alistair Docherty,-14
3,Daniel Hudson,-13
4,Sam Saunders,-12


In [11]:
if post_cut:
    #replace name with what appears in DK
    #To make sure names match, replace all names in df_odds with actual DK names in df
    from difflib import get_close_matches
    #df['Name'] = ""
    for i in np.arange(len(df_made_cut)):
        curr_name = df_made_cut['Name'].iloc[i]
        try:
            new_name = get_close_matches(curr_name,df['Name'],cutoff=.8)[0]
            if new_name != curr_name:
                print("{} replaced with {}".format(curr_name,new_name))
        except:
            new_name = 'No Name'
            print('NO MATCH: curr_name = {}, new_name = {}'.format(curr_name,new_name))
        df_made_cut['Name'].iloc[i] = new_name
    #df2['Name'] = df2['Name'].apply(lambda x: get_close_matches(x,df2_sal['Name']))
    print(df_made_cut.head(5))

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Sam Saunders replaced with Samuel Saunders
Trevor Lampson (a) replaced with Trevor Lampson
                Name  Score
0     Tyler Weworski    -16
1    Sahith Theegala    -15
2  Alistair Docherty    -14
3      Daniel Hudson    -13
4    Samuel Saunders    -12


In [12]:
made_cut_names = list(df_made_cut.Name.dropna())
print(len(made_cut_names))
made_cut_names[:5]

24


['Tyler Weworski',
 'Sahith Theegala',
 'Alistair Docherty',
 'Daniel Hudson',
 'Samuel Saunders']

In [13]:
len(df[df.Name.isin(made_cut_names)])

24

### Add odds

Manually create csv from https://smartgolfbets.com/tournament-sheets/, label name column as "Name" to match DK format, and only keep columns with Odds, e.g. Odds1, Odds2

In [14]:
# df_odds = pd.read_csv('Data/Outlaw_odds_20200413.csv')
# df_odds.rename(columns={'Odds1':'Odds'},inplace=True)
df_odds = pd.read_csv('Data/Outlaw_odds_20200601.csv')
df_odds.iloc[np.r_[0:4,-4:0]]

,Name,Odds
0,Brady Calkins,7.0
1,Sahith Theegala,7.5
2,Tyler Weworski,9.5
3,Nicolo Galletti,11.0
43,Anthony Quezada,401.0
44,Matt Lutz,401.0
45,Adam Petrocelli,751.0
46,Alex Trevino,751.0


In [15]:
#df_odds.Odds = df_odds.Odds.str.replace(',','').astype(float)
df_odds.Odds = df_odds.Odds.astype(str).str.replace(',','').astype(float)
df_odds.iloc[np.r_[0:2,-2:0]]

,Name,Odds
0,Brady Calkins,7.0
1,Sahith Theegala,7.5
45,Adam Petrocelli,751.0
46,Alex Trevino,751.0


In [16]:
#strip trailing whiktespace
df_odds.Name.iloc[0]

'Brady Calkins'

In [17]:
df_odds.Name = df_odds.Name.str.rstrip()
df_odds.Name.iloc[0]

'Brady Calkins'

In [18]:
#Change to regular odds
if df_odds.Odds.min()>100:
    df_odds.Odds = df_odds.Odds / 100
df_odds.head(3)

,Name,Odds
0,Brady Calkins,7.0
1,Sahith Theegala,7.5
2,Tyler Weworski,9.5


In [19]:
#If anyone is repeated, just keep their higher odds
df_odds = df_odds.groupby('Name')['Odds'].max().reset_index().dropna()
#df_odds.rename(columns={'Odds2':'Odds'},inplace=True)
df_odds.head()

,Name,Odds
0,Adam Petrocelli,751.0
1,Alex Lobeck,151.0
2,Alex Trevino,751.0
3,Alistair Docherty,12.0
4,Andrew Funk,19.0


In [20]:
df_odds[df_odds.Odds.isnull()]

,Name,Odds


In [21]:
#To make sure names match, replace all names in df_odds with actual DK names in df
from difflib import get_close_matches
#df['Name'] = ""
for i in np.arange(len(df_odds)):
    curr_name = df_odds['Name'].iloc[i]
    try:
        new_name = get_close_matches(curr_name,df['Name'],cutoff=.75)[0]
        if new_name != curr_name:
            print("{} replaced with {}".format(curr_name,new_name))
    except:
        new_name = 'No Name'
        print('NO MATCH: curr_name = {}, new_name = {}'.format(curr_name,new_name))
    df_odds['Name'].iloc[i] = new_name
#df2['Name'] = df2['Name'].apply(lambda x: get_close_matches(x,df2_sal['Name']))
df_odds.head(5)

NO MATCH: curr_name = Adam Petrocelli, new_name = No Name
NO MATCH: curr_name = Alex Lobeck, new_name = No Name
NO MATCH: curr_name = Anthony Quezada, new_name = No Name
NO MATCH: curr_name = Barry Enright, new_name = No Name
NO MATCH: curr_name = Caleb Ramirez, new_name = No Name
NO MATCH: curr_name = Chris Korte, new_name = No Name
NO MATCH: curr_name = Derek Ackerman, new_name = No Name
NO MATCH: curr_name = Jack McDevitt, new_name = No Name
NO MATCH: curr_name = Jacob Loya, new_name = No Name
NO MATCH: curr_name = Jeff Nichols, new_name = No Name
NO MATCH: curr_name = Marty Sanchez, new_name = No Name
NO MATCH: curr_name = Matt McClung, new_name = No Name
NO MATCH: curr_name = Matthew Snyder, new_name = No Name
NO MATCH: curr_name = Michael McGowan, new_name = No Name
NO MATCH: curr_name = Mitchel Carlson, new_name = No Name
NO MATCH: curr_name = Noah Hofman, new_name = No Name
NO MATCH: curr_name = Oliver Roberts, new_name = No Name
NO MATCH: curr_name = RJ Wood, new_name = No Nam

,Name,Odds
0,No Name,751.0
1,No Name,151.0
2,Alex Trevino,751.0
3,Alistair Docherty,12.0
4,Andrew Funk,19.0


In [22]:
df[df.Name.str.contains('Doyle')]

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame


In [23]:
#Add odds to df
df_all = pd.merge(df,df_odds,how='left')
df_all.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,Odds
0,G,Sahith Theegala (14591201),Sahith Theegala,14591201,G,10400,Lone Tree Classic,Golf,0,7.5
1,G,Alistair Docherty (14591202),Alistair Docherty,14591202,G,10200,Lone Tree Classic,Golf,0,12.0
2,G,Brady Calkins (14591203),Brady Calkins,14591203,G,10000,Lone Tree Classic,Golf,0,7.0
3,G,Tyler Weworski (14591204),Tyler Weworski,14591204,G,9700,Lone Tree Classic,Golf,0,9.5
4,G,Nicolo Galletti (14591205),Nicolo Galletti,14591205,G,9400,Lone Tree Classic,Golf,0,11.0


In [24]:
len(df_all[df_all.Name.isin(made_cut_names)])

24

In [25]:
df_all[df_all.Name.isin(made_cut_names)].tail()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,Odds
19,G,Chris Figueroa (14591220),Chris Figueroa,14591220,G,6400,Lone Tree Classic,Golf,0,81.0
20,G,Mitchell Homb (14591221),Mitchell Homb,14591221,G,6300,Lone Tree Classic,Golf,0,141.0
21,G,Roy Cootes (14591222),Roy Cootes,14591222,G,6200,Lone Tree Classic,Golf,0,301.0
22,G,Alex Trevino (14591223),Alex Trevino,14591223,G,6100,Lone Tree Classic,Golf,0,751.0
23,G,Matt Lutz (14591224),Matt Lutz,14591224,G,6000,Lone Tree Classic,Golf,0,401.0


In [26]:
#See if anyone has MADE CUT but has NO ODDS
df_all[(df_all.Odds.isnull())&(df_all.Name.isin(made_cut_names))]

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,Odds
14,G,Brandon Bauman (14591215),Brandon Bauman,14591215,G,7100,Lone Tree Classic,Golf,0,NaN


In [27]:
df_all['Win'] = 1 / (df_all.Odds+1)
df_all.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,Odds,Win
0,G,Sahith Theegala (14591201),Sahith Theegala,14591201,G,10400,Lone Tree Classic,Golf,0,7.5,0.117647
1,G,Alistair Docherty (14591202),Alistair Docherty,14591202,G,10200,Lone Tree Classic,Golf,0,12.0,0.076923
2,G,Brady Calkins (14591203),Brady Calkins,14591203,G,10000,Lone Tree Classic,Golf,0,7.0,0.125000
3,G,Tyler Weworski (14591204),Tyler Weworski,14591204,G,9700,Lone Tree Classic,Golf,0,9.5,0.095238
4,G,Nicolo Galletti (14591205),Nicolo Galletti,14591205,G,9400,Lone Tree Classic,Golf,0,11.0,0.083333


In [28]:
df_all.iloc[np.r_[0:6,-3:0]]

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,Odds,Win
0,G,Sahith Theegala (14591201),Sahith Theegala,14591201,G,10400,Lone Tree Classic,Golf,0,7.5,0.117647
1,G,Alistair Docherty (14591202),Alistair Docherty,14591202,G,10200,Lone Tree Classic,Golf,0,12.0,0.076923
2,G,Brady Calkins (14591203),Brady Calkins,14591203,G,10000,Lone Tree Classic,Golf,0,7.0,0.125000
3,G,Tyler Weworski (14591204),Tyler Weworski,14591204,G,9700,Lone Tree Classic,Golf,0,9.5,0.095238
4,G,Nicolo Galletti (14591205),Nicolo Galletti,14591205,G,9400,Lone Tree Classic,Golf,0,11.0,0.083333
5,G,Andrew Funk (14591206),Andrew Funk,14591206,G,9200,Lone Tree Classic,Golf,0,19.0,0.050000
21,G,Roy Cootes (14591222),Roy Cootes,14591222,G,6200,Lone Tree Classic,Golf,0,301.0,0.003311
22,G,Alex Trevino (14591223),Alex Trevino,14591223,G,6100,Lone Tree Classic,Golf,0,751.0,0.001330
23,G,Matt Lutz (14591224),Matt Lutz,14591224,G,6000,Lone Tree Classic,Golf,0,401.0,0.002488


In [29]:
len(df_all[df_all.Name.isin(made_cut_names)])

24

In [30]:
df_all = df_all[~df_all.Odds.isnull()]

### Add projected points per trained polynomial model

In [31]:
#cap odds at 1000
df_all.Odds = np.where(df_all.Odds>1000,
                       1000,
                      df_all.Odds)

In [32]:
#fitted model
import pickle
pkl_filename = 'outlaw_proj_model.pkl'
outlaw_proj_model_pkl = open(pkl_filename, 'rb')
outlaw_proj_model = pickle.load(outlaw_proj_model_pkl)
outlaw_proj_model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [33]:
from sklearn.preprocessing import PolynomialFeatures  
poly = PolynomialFeatures(degree = 4) 
df_all['Points'] = outlaw_proj_model.predict(poly.fit_transform(df_all[['Odds']]))
df_all.iloc[np.r_[0:4,-4:0]]

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,Odds,Win,Points
0,G,Sahith Theegala (14591201),Sahith Theegala,14591201,G,10400,Lone Tree Classic,Golf,0,7.5,0.117647,7.414324
1,G,Alistair Docherty (14591202),Alistair Docherty,14591202,G,10200,Lone Tree Classic,Golf,0,12.0,0.076923,7.142664
2,G,Brady Calkins (14591203),Brady Calkins,14591203,G,10000,Lone Tree Classic,Golf,0,7.0,0.125000,7.444730
3,G,Tyler Weworski (14591204),Tyler Weworski,14591204,G,9700,Lone Tree Classic,Golf,0,9.5,0.095238,7.293144
20,G,Mitchell Homb (14591221),Mitchell Homb,14591221,G,6300,Lone Tree Classic,Golf,0,141.0,0.007042,0.749564
21,G,Roy Cootes (14591222),Roy Cootes,14591222,G,6200,Lone Tree Classic,Golf,0,301.0,0.003311,-4.158537
22,G,Alex Trevino (14591223),Alex Trevino,14591223,G,6100,Lone Tree Classic,Golf,0,751.0,0.001330,-8.811721
23,G,Matt Lutz (14591224),Matt Lutz,14591224,G,6000,Lone Tree Classic,Golf,0,401.0,0.002488,-6.012670


In [34]:
df_all=df_all[['Name + ID',
              'Name',
              'Salary',
              'Odds',
              'Win',
              'Points']]
df_all.head()

,Name + ID,Name,Salary,Odds,Win,Points
0,Sahith Theegala (14591201),Sahith Theegala,10400,7.5,0.117647,7.414324
1,Alistair Docherty (14591202),Alistair Docherty,10200,12.0,0.076923,7.142664
2,Brady Calkins (14591203),Brady Calkins,10000,7.0,0.125000,7.444730
3,Tyler Weworski (14591204),Tyler Weworski,9700,9.5,0.095238,7.293144
4,Nicolo Galletti (14591205),Nicolo Galletti,9400,11.0,0.083333,7.202724


In [35]:
len(df_all)

23

### Add tee times and adjusted points
Adjust by -0.64 pts per hr

In [36]:
df_tees = pd.read_csv('Data/golf_tee_times_20200526.csv',encoding='latin')
df_tees.head(8)

,Name,Tee,Round,Pairing
0,BA,NaN,1.0,"Blake Allen, Austin Bautista, Perry Cohen"
1,Blake Allen,9:27 AM,NaN,NaN
2,"Lenexa, KS",NaN,NaN,NaN
3,ZA,NaN,1.0,"Mitchel Carlson, Luke Kristo, Zach Allred"
4,Zach Allred,12:51 PM,NaN,NaN
5,"Colorado Springs, CO",NaN,NaN,NaN
6,JA,NaN,1.0,"Matthew Liringis, John Greco, James Andrews"
7,James Andrews,9:36 AM,NaN,NaN


In [37]:
df_tees2 = df_tees[['Name','Tee']].dropna().reset_index(drop=True)
df_tees2.iloc[np.r_[0:5,-5:0]]

,Name,Tee
0,Blake Allen,9:27 AM
1,Zach Allred,12:51 PM
2,James Andrews,9:36 AM
3,Nick Arman,12:09 PM
4,Peter Badawy,9:09 AM
69,Tyler Weworski,12:33 PM
70,Riley Wheeldon,11:00 AM
71,Blake Woods,10:42 AM
72,Norman Xiong,12:24 PM
73,Colton Yates,11:18 AM


In [38]:
#replace names with what appears in df_all
for i in np.arange(len(df_tees2)):
    curr_name = df_tees2['Name'].iloc[i]
    try:
        new_name = get_close_matches(curr_name,df_all['Name'],cutoff=.85)[0]
        if new_name != curr_name:
            _
            print("{} replaced with {}".format(curr_name,new_name))
    except:
        new_name = 'No Name'
        #print('NO MATCH: curr_name = {}, new_name = {}'.format(curr_name,new_name))
    df_tees2['Name'].iloc[i] = new_name
#df2['Name'] = df2['Name'].apply(lambda x: get_close_matches(x,df2_sal['Name']))
df_tees2.head(5)

Sam Saunders replaced with Samuel Saunders


,Name,Tee
0,Blake Allen,9:27 AM
1,No Name,12:51 PM
2,No Name,9:36 AM
3,No Name,12:09 PM
4,No Name,9:09 AM


In [39]:
tee_times=dict(zip(df_tees2.Name,
                  df_tees2.Tee))
df_all['Tee']=df_all.Name.map(tee_times)
df_all.head(11)

,Name + ID,Name,Salary,Odds,Win,Points,Tee
0,Sahith Theegala (14591201),Sahith Theegala,10400,7.5,0.117647,7.414324,NaN
1,Alistair Docherty (14591202),Alistair Docherty,10200,12.0,0.076923,7.142664,11:18 AM
2,Brady Calkins (14591203),Brady Calkins,10000,7.0,0.125000,7.444730,9:18 AM
3,Tyler Weworski (14591204),Tyler Weworski,9700,9.5,0.095238,7.293144,12:33 PM
4,Nicolo Galletti (14591205),Nicolo Galletti,9400,11.0,0.083333,7.202724,11:18 AM
5,Andrew Funk (14591206),Andrew Funk,9200,19.0,0.050000,6.727161,11:00 AM
6,Neil Johnson (14591207),Neil Johnson,9000,15.0,0.062500,6.963542,12:00 PM
7,Mark Baldwin (14591208),Mark Baldwin,8700,12.0,0.076923,7.142664,NaN
8,Samuel Saunders (14591209),Samuel Saunders,8400,65.0,0.015152,4.202951,12:24 PM
9,Trevor Lampson (14591210),Trevor Lampson,8100,21.0,0.045455,6.610014,NaN


In [40]:
t =pd.to_datetime(df_all.Tee)
td_ser = t-t.min()
df_all['tee_delay'] = [td.seconds/3600 for td in td_ser]
df_all.head()

,Name + ID,Name,Salary,Odds,Win,Points,Tee,tee_delay
0,Sahith Theegala (14591201),Sahith Theegala,10400,7.5,0.117647,7.414324,NaN,NaN
1,Alistair Docherty (14591202),Alistair Docherty,10200,12.0,0.076923,7.142664,11:18 AM,2.00
2,Brady Calkins (14591203),Brady Calkins,10000,7.0,0.125000,7.444730,9:18 AM,0.00
3,Tyler Weworski (14591204),Tyler Weworski,9700,9.5,0.095238,7.293144,12:33 PM,3.25
4,Nicolo Galletti (14591205),Nicolo Galletti,9400,11.0,0.083333,7.202724,11:18 AM,2.00


In [41]:
#Add adjusted points
pts_per_hr = 0
df_all['Points_adj'] = df_all.Points - pts_per_hr*df_all.tee_delay
df_all.head()

,Name + ID,Name,Salary,Odds,Win,Points,Tee,tee_delay,Points_adj
0,Sahith Theegala (14591201),Sahith Theegala,10400,7.5,0.117647,7.414324,NaN,NaN,NaN
1,Alistair Docherty (14591202),Alistair Docherty,10200,12.0,0.076923,7.142664,11:18 AM,2.00,7.142664
2,Brady Calkins (14591203),Brady Calkins,10000,7.0,0.125000,7.444730,9:18 AM,0.00,7.444730
3,Tyler Weworski (14591204),Tyler Weworski,9700,9.5,0.095238,7.293144,12:33 PM,3.25,7.293144
4,Nicolo Galletti (14591205),Nicolo Galletti,9400,11.0,0.083333,7.202724,11:18 AM,2.00,7.202724


### Now remove anyone cut

In [42]:
if post_cut:
    #made_cut_list = made_cut.Name.tolist()
    df_all = df_all[df_all.Name.isin(made_cut_names)]
    print(len(df_all))
    print(set(made_cut_names) - set(df.Name))
    df_all
    

23
set()


In [43]:
set(made_cut_names) - set(df_all.Name)

{'Brandon Bauman'}

In [44]:
df_all

,Name + ID,Name,Salary,Odds,Win,Points,Tee,tee_delay,Points_adj
0,Sahith Theegala (14591201),Sahith Theegala,10400,7.5,0.117647,7.414324,NaN,NaN,NaN
1,Alistair Docherty (14591202),Alistair Docherty,10200,12.0,0.076923,7.142664,11:18 AM,2.00,7.142664
2,Brady Calkins (14591203),Brady Calkins,10000,7.0,0.125000,7.444730,9:18 AM,0.00,7.444730
3,Tyler Weworski (14591204),Tyler Weworski,9700,9.5,0.095238,7.293144,12:33 PM,3.25,7.293144
4,Nicolo Galletti (14591205),Nicolo Galletti,9400,11.0,0.083333,7.202724,11:18 AM,2.00,7.202724
5,Andrew Funk (14591206),Andrew Funk,9200,19.0,0.050000,6.727161,11:00 AM,1.70,6.727161
6,Neil Johnson (14591207),Neil Johnson,9000,15.0,0.062500,6.963542,12:00 PM,2.70,6.963542
7,Mark Baldwin (14591208),Mark Baldwin,8700,12.0,0.076923,7.142664,NaN,NaN,NaN
8,Samuel Saunders (14591209),Samuel Saunders,8400,65.0,0.015152,4.202951,12:24 PM,3.10,4.202951
9,Trevor Lampson (14591210),Trevor Lampson,8100,21.0,0.045455,6.610014,NaN,NaN,NaN


# Simulate to find gow often players are in optimal

In [45]:
df_all = df_all.reset_index(drop=True)
df_all.iloc[np.r_[0:3,-3:0]]

,Name + ID,Name,Salary,Odds,Win,Points,Tee,tee_delay,Points_adj
0,Sahith Theegala (14591201),Sahith Theegala,10400,7.5,0.117647,7.414324,NaN,NaN,NaN
1,Alistair Docherty (14591202),Alistair Docherty,10200,12.0,0.076923,7.142664,11:18 AM,2.0,7.142664
2,Brady Calkins (14591203),Brady Calkins,10000,7.0,0.125000,7.444730,9:18 AM,0.0,7.444730
20,Roy Cootes (14591222),Roy Cootes,6200,301.0,0.003311,-4.158537,NaN,NaN,NaN
21,Alex Trevino (14591223),Alex Trevino,6100,751.0,0.001330,-8.811721,NaN,NaN,NaN
22,Matt Lutz (14591224),Matt Lutz,6000,401.0,0.002488,-6.012670,10:18 AM,1.0,-6.012670


In [49]:
df_all.Salary.mean()

7878.260869565217

In [50]:
df_all.Points.mean()

4.042208896885227

In [46]:
df_now=df_all.copy()
df_now['sim_pts']=df_now.Points+10.8*np.random.randn(len(df_now))
df_now[['Name','Points','sim_pts']].head()

,Name,Points,sim_pts
0,Sahith Theegala,7.414324,-12.899116
1,Alistair Docherty,7.142664,-3.227815
2,Brady Calkins,7.444730,-14.808651
3,Tyler Weworski,7.293144,-1.426157
4,Nicolo Galletti,7.202724,-4.419081


In [47]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 9 columns):
Name + ID     23 non-null object
Name          23 non-null object
Salary        23 non-null int64
Odds          23 non-null float64
Win           23 non-null float64
Points        23 non-null float64
Tee           11 non-null object
tee_delay     11 non-null float64
Points_adj    11 non-null float64
dtypes: float64(5), int64(1), object(3)
memory usage: 1.7+ KB


In [48]:
def sim_optimal_team(df,sd=10.8,points_col='Points'):
    #df needs column called Points
    # sd is standard dev for added normal points.  Set lower for less shuffling of teams away from pure max expectation
    #sd = 10.8
    df_now = df.copy()
    df_now['sim_pts']=df[points_col]+sd*np.random.randn(len(df_now))
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    sim_pts = df_now.sim_pts.values

    cap = 50000
    constraints = [cp.sum(x) == 6,
             x*s <= cap]
    obj = cp.Maximize(x*sim_pts)
    prob = cp.Problem(obj, constraints)
    prob.solve()  
    opt_team = df_now['Name'][x.value>.5].tolist()
    return opt_team

sim_optimal_team(df_all)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/cvxpy/expressions/expression.py:516: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/cvxpy/expressions/expression.py:516: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


SolverError: 

                    CVXPY needs additional software (a `mixed-integer solver`) to handle this model.
                    The web documentation
                        https://www.cvxpy.org/tutorial/advanced/index.html#mixed-integer-programs
                    reviews open-source and commercial options for mixed-integer solvers.

                    Quick fix: if you install the python package CVXOPT (pip install cvxopt),
                    then CVXPY can use the open-source mixed-integer solver `GLPK`.
                

In [ ]:
df_all.head()

In [ ]:
num_sims = 2000
total_sal=0
df_all['num_opt']=0
df_for_sim = df_all[df_all.Salary>=6200]
df_opt_teams = pd.Series()
for i in range(num_sims):
    if i%(int(num_sims/4))==0:
        print('i = {}'.format(i))
    opt_team=sim_optimal_team(df_all,points_col='Points')
    opt_sal=df_all[df_all.Name.isin(opt_team)].Salary.sum()
    total_sal+=opt_sal
    df_all.num_opt=np.where(df_all.Name.isin(opt_team),
                           df_all.num_opt +1,df_all.num_opt)
    df_opt_teams.loc[len(df_opt_teams)] = ", ".join(opt_team)

print(total_sal/num_sims)
df_all.sort_values(by='num_opt',
                   ascending=False).head(15)

In [ ]:
to_remove = []
df_now = df_all[~df_all.Name.isin(to_remove)].sort_values(by='num_opt',
                   ascending=False).reset_index(drop=True)
df_now.head(25)

In [ ]:
df_now.head(15).Salary.mean()

### Try to build teams

In [ ]:
numteams=37


# Now Optimize in different ways

In [ ]:
to_remove = ['Kaleb Gorbahn','Ryan Gronlund','Kirk Triplett']
to_remove = []

In [ ]:
#Max Win prob
df_now = df_all.copy()
min_sal = 6000
max_sal = 12000
df_now = df_now[df_now.Salary>=min_sal]
n = len(df_now)
x = cp.Variable(n,boolean=True)
s = df_now.Salary.values
w = df_now.Win.values
o = df_now.Odds.values

# c = df_now.Cut.values
# T20 = df_now['Top 20'].values
# own = df_now.Ownership.values
cap = 50000
constraints = [cp.sum(x) == 6,
             x*s <= cap]

# Form objective.
#Max Win probability:
obj = cp.Maximize(x*w)

#Minimize Winning Odds
#obj = cp.Minimize(x*o)

# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()  # Returns the optimal value
opt_team = df_now['Name'][x.value>.5].tolist()
sal = df_now[df_now.Name.isin(opt_team)].Salary.sum()
print(sal,opt_team)

In [ ]:
#Max Win prob, top n teams
#df = df_all.copy()
df_now = df_all.copy()
min_sal = 7000
max_sal = 99900
df_now = df_now[df_now.Salary>=min_sal]
df_now = df_now[~df_now.Name.isin(to_remove)]

#Filter down remaining players by sal
df_now = df_now[(df_now.Salary>=min_sal) &
               (df_now.Salary<=max_sal)]
# df_now = df_now[~df_now.Player.isin(to_remove)]
numteams = 4
opt_teams_win = []
for i in np.arange(numteams): #number of teams
#     print(df_now.head())
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    w = df_now.Win.values
#     c = df_now.Cut.values
#     T20 = df_now['Top 20'].values
#     own = df_now.Ownership.values
#     cap = sal_left
    constraints = [cp.sum(x) == 6,
                 x*s <= 50000]

    # Form objective.
    obj = cp.Maximize(x*w)

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value
    opt_team = df_now['Name'][x.value>.5].tolist()
    sal = df_now[df_now.Name.isin(opt_team)].Salary.sum()
    print('Team {} (obj = {:.2f}, sal = {}): '.format(i+1,prob.value,sal),opt_team)
    opt_teams_win.append(opt_team)
    #Remove team and go again
    df_now = df_now[~ df_now.Name.isin(opt_team)]

In [ ]:
df_all.head(9)

In [ ]:
#Min total odds
#df = df_all.copy()
df_now = df_all.copy()
min_sal = 6200
max_sal = 12000
df_now = df_now[df_now.Salary>=min_sal]

#Filter down remaining players by sal
df_now = df_now[(df_now.Salary>=min_sal) &
               (df_now.Salary<=max_sal)]
#to_remove = ['Matt Gurska']
#to_remove =[]
df_now = df_now[~df_now.Name.isin(to_remove)]
# df_now = df_now[~df_now.Player.isin(to_remove)]
numteams = 5
opt_teams_odds = []
for i in np.arange(numteams): #number of teams
#     print(df_now.head())
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    w = df_now.Win.values
    o = df_now.Odds.values
#     c = df_now.Cut.values
#     T20 = df_now['Top 20'].values
#     own = df_now.Ownership.values
#     cap = sal_left
    constraints = [cp.sum(x) == 6,
                 x*s <= 50000]

    # Form objective.
    #obj = cp.Maximize(x*w)
    obj = cp.Minimize(x*o)

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value
    opt_team = df_now['Name'][x.value>.5].tolist()
    sal = df_now[df_now.Name.isin(opt_team)].Salary.sum()
    print('Team {} (obj = {:.2f}, sal = {}): '.format(i+1,prob.value,sal),opt_team)
    opt_teams_odds.append(opt_team)
    #Remove team and go again
    df_now = df_now[~ df_now.Name.isin(opt_team)]

In [ ]:
#Projected points (points gained)
#df = df_all.copy()
df_now = df_all.copy()
min_sal = 6200
max_sal = 99900
df_now = df_now[df_now.Salary>=min_sal]
#to_remove = ['Matt Gurska']
#to_remove =[]
df_now = df_now[~df_now.Name.isin(to_remove)]

#Filter down remaining players by sal
df_now = df_now[(df_now.Salary>=min_sal) &
               (df_now.Salary<=max_sal)]

# df_now = df_now[~df_now.Player.isin(to_remove)]
numteams = 5
opt_teams_pts = []
for i in np.arange(numteams): #number of teams
#     print(df_now.head())
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    w = df_now.Win.values
    p = df_now.Points.values
#     c = df_now.Cut.values
#     T20 = df_now['Top 20'].values
#     own = df_now.Ownership.values
#     cap = sal_left
    constraints = [cp.sum(x) == 6,
                 x*s <= 50000]

    # Form objective.
    obj = cp.Maximize(x*p)

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value
    opt_team = df_now['Name'][x.value>.5].tolist()
    sal = df_now[df_now.Name.isin(opt_team)].Salary.sum()
    print('Team {} (obj = {:.2f}, sal = {}): '.format(i+1,prob.value,sal),opt_team)
    opt_teams_pts.append(opt_team)
    #Remove team and go again
    df_now = df_now[~ df_now.Name.isin(opt_team)]

In [ ]:
#In one not the other
for i in range(numteams):
    print('In top {} teams:'.format(i+1))
    print('  In Odds, not Pts:',set(np.hstack(opt_teams_win[:i+1]))-set(np.hstack(opt_teams_pts[:i+1])))
    print('  In Pts, not Odds:',set(np.hstack(opt_teams_pts[:i+1]))-set(np.hstack(opt_teams_win[:i+1])))

In [ ]:
#In one not the other
for i in range(numteams):
    print('In top {} teams:'.format(i+1))
    print('  In Odds, not Win:',set(np.hstack(opt_teams_odds[:i+1]))-set(np.hstack(opt_teams_win[:i+1])))
    print('  In Win, not Odds:',set(np.hstack(opt_teams_win[:i+1]))-set(np.hstack(opt_teams_odds[:i+1])))

In [ ]:
df_all.sort_values(by='num_opt',ascending=False).reset_index(drop=True).head(25)

## See who underperformed previous day

In [ ]:
df_results = pd.read_csv('Data/outlaw_results_20200601.csv')
df_results=df_results[['Player','FPTS']]
df_results.rename(columns={'Player':'Name'},inplace=True)
df_results['FPTS_gained'] = df_results.FPTS - df_results.FPTS.mean()
df_results.head()

In [ ]:
#Compare to projections
df_results = pd.merge(df_results,df_all[['Name','Points']],how='left')
df_results['Diff'] = df_results.FPTS_gained - df_results.Points
df_results = df_results.sort_values(by='Diff',ascending=True)
df_results.head(15)

### Set final optimal teams

In [62]:
df_now = df_all.copy()
lock =['Eddie Olson','Dylan Wu']
lock = ['Trevor Lampson']
lock = []
sal_used = df_now[df_now.Name.isin(lock)].Salary.sum()
print (sal_used)
players_left = 6 - len(lock)
lock_full_names = df_now[df_now.Name.isin(lock)]['Name + ID'].tolist()
lock_full_names

0


[]

In [ ]:
min_sal = 6600
max_sal = 99900
df_now = df_all.copy()
df_now = df_now[(df_now.Salary>=min_sal) &
               (df_now.Salary<=max_sal)]
df_now = df_now[~df_now.Name.isin(lock)]
to_remove = ['Mitchel Carlson','Derek Ackerman','Matt McClung','Alex Trevino','Jack McDevitt',
            'Ollie Roberts','Chris Korte']
#to_remove =[]
df_now = df_now[~df_now.Name.isin(to_remove)]
#df_now.Points = np.where(df_now.Wave=='AM',df_now.Points+am_bump_pts,df_now.Points)

numteams = 5
opt_teams_odds_lock = []
for i in np.arange(numteams): #number of teams
#     print(df_now.head())
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    w = df_now.Win.values
    o = df_now.Odds.values
    p = df_now.Points.values
    p2 = df_now.Points_adj.values
#     c = df_now.Cut.values
#     T20 = df_now['Top 20'].values
#     own = df_now.Ownership.values
    cap = 50000 - sal_used
    constraints = [cp.sum(x) == players_left,
                 x*s <= cap]

    # Form objective.
    obj = cp.Maximize(x*w)
    #obj = cp.Minimize(x*o)

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value
    opt_team = df_now['Name'][x.value>.5].tolist()
    sal = df_now[df_now.Name.isin(opt_team)].Salary.sum()
    print('Team {} (obj = {:.2f}, sal = {}): '.format(i+1,prob.value,sal+sal_used),
          opt_team)
    opt_teams_odds_lock.append(opt_team)
    #Remove team and go again
    df_now = df_now[~ df_now.Name.isin(opt_team)]

In [51]:
df_all

,Name + ID,Name,Salary,Odds,Win,Points,Tee,tee_delay,Points_adj
0,Sahith Theegala (14591201),Sahith Theegala,10400,7.5,0.117647,7.414324,NaN,NaN,NaN
1,Alistair Docherty (14591202),Alistair Docherty,10200,12.0,0.076923,7.142664,11:18 AM,2.00,7.142664
2,Brady Calkins (14591203),Brady Calkins,10000,7.0,0.125000,7.444730,9:18 AM,0.00,7.444730
3,Tyler Weworski (14591204),Tyler Weworski,9700,9.5,0.095238,7.293144,12:33 PM,3.25,7.293144
4,Nicolo Galletti (14591205),Nicolo Galletti,9400,11.0,0.083333,7.202724,11:18 AM,2.00,7.202724
5,Andrew Funk (14591206),Andrew Funk,9200,19.0,0.050000,6.727161,11:00 AM,1.70,6.727161
6,Neil Johnson (14591207),Neil Johnson,9000,15.0,0.062500,6.963542,12:00 PM,2.70,6.963542
7,Mark Baldwin (14591208),Mark Baldwin,8700,12.0,0.076923,7.142664,NaN,NaN,NaN
8,Samuel Saunders (14591209),Samuel Saunders,8400,65.0,0.015152,4.202951,12:24 PM,3.10,4.202951
9,Trevor Lampson (14591210),Trevor Lampson,8100,21.0,0.045455,6.610014,NaN,NaN,NaN


In [55]:
select_team = ['Sahith Theegala','Alistair Docherty','Tyler Weworski','Nicolo Galletti',
               'Neil Johnson','Trevor Lampson','Daniel Hudson','Steven Alker','Zander Winston',
               'Jino Sohn','Blake Allen','Sam Gillis','Chris Figueroa','Mitchell Homb']
#select_team = []

## Generate lineups!

In [ ]:
### Assign exposures for top j teams
num_teams_deep = 5
own_gap = 5
exposures = list(np.arange(50,50-own_gap*num_teams_deep,-own_gap))
if len(exposures)<len(opt_teams_odds_lock):
    exposures = exposures + [0]*(len(opt_teams_odds_lock)-len(exposures))
exposures

In [ ]:
#Create df to hold player pool
df_teams = df_all[['Name + ID','Name','Salary']].drop_duplicates()
df_teams.sort_values(by='Salary',ascending=False,inplace=True)
df_teams['Exposure'] = 0
df_teams['Actual Exposure'] = 0
df_teams.head()

#Add in exposures for players in player pool
for i in range(num_teams_deep):
    team = opt_teams_odds_lock[i]
    df_teams.Exposure = np.where(df_teams.Name.isin(team),exposures[i],df_teams.Exposure)
    
df_teams = df_teams[df_teams.Exposure > 0].reset_index(drop=True)
df_teams

In [ ]:
#Get list of all possible additional players that aren't already in the team
df_more = df_all[~df_all.Name.isin(df_teams.Name.tolist())].sort_values(by='num_opt',ascending=False)
df_more = df_more[~df_more.Name.isin(lock)]
df_more = df_more[~df_more.Name.isin(to_remove)]
df_more = df_more[(df_more.Salary<12000)&
                 (df_more.Salary>=200)]
df_more = df_more[['Name + ID','Name','Salary']]
df_more['Exposure'] = df_teams.Exposure.min()-6
df_more['Actual Exposure'] = 0
df_more.head(20)

In [ ]:
#Option 1: Add specific names
more_names = ['Zachary Gaugert','Dalen Yamauchi','Dylan Wu']
more_names = []
if (len(more_names)>0):
    df_more_now = df_more[df_more.Name.isin(more_names)]
    df_teams = pd.concat([df_teams,df_more_now]).sort_values(by='Salary',ascending=False).reset_index(drop=True)

In [ ]:
df_more.head()

In [ ]:
#Option 2: Add players in top N Win teams
#First take a peak at who this would add
num_win_teams = 1
more_team_by_win = list(set(np.hstack(opt_teams_win[:num_win_teams]))-set(df_teams.Name.tolist()))
print(opt_teams_win[:num_win_teams])
print('Will add: ',more_team_by_win)

In [ ]:
df_more.head(20)

In [ ]:
add_win =False
if add_win:
    df_more_now = df_more[df_more.Name.isin(more_team_by_win)]
    df_teams = pd.concat([df_teams,df_more_now]).sort_values(by='Salary',ascending=False).reset_index(drop=True)
    df_teams

In [ ]:
# Option 3: Add top n most likely to appera in optimal
num_add = 0
max_sal_add = 99900
if num_add>0:
    df_more_now = df_more[df_more.Salary<=max_sal_add].iloc[:num_add]
    print(df_more_now)
    df_teams = pd.concat([df_teams,df_more_now]).sort_values(by='Salary',ascending=False).reset_index(drop=True)

In [ ]:
# Option 3: Add top n most likely to appera in optimal
num_add = 0
max_sal_add = 7300
if num_add>0:
    df_more_now = df_more[df_more.Salary<=max_sal_add].iloc[:num_add]
    print(df_more_now)
    df_teams = pd.concat([df_teams,df_more_now]).sort_values(by='Salary',ascending=False).reset_index(drop=True)

In [ ]:
#Remove anyone?
remove_from_teams = ['Neil Tillman']
remove_from_teams = []
df_teams = df_teams[~df_teams.Name.isin(remove_from_teams)]


In [56]:
if len(select_team)>0:
    df_teams = df_all[df_all.Name.isin(select_team)][['Name + ID','Name','Salary']].drop_duplicates()
    df_teams.sort_values(by='Salary',ascending=False,inplace=True)
    df_teams['Exposure'] = 50
    df_teams['Actual Exposure'] = 0

    df_teams = df_teams[df_teams.Exposure > 0].reset_index(drop=True)

In [57]:
#Now see what final df_teams looks like
df_teams = df_teams.drop_duplicates().reset_index(drop=True)
df_teams.Exposure = df_teams.Exposure.fillna(35)
df_teams

,Name + ID,Name,Salary,Exposure,Actual Exposure
0,Sahith Theegala (14591201),Sahith Theegala,10400,50,0
1,Alistair Docherty (14591202),Alistair Docherty,10200,50,0
2,Tyler Weworski (14591204),Tyler Weworski,9700,50,0
3,Nicolo Galletti (14591205),Nicolo Galletti,9400,50,0
4,Neil Johnson (14591207),Neil Johnson,9000,50,0
5,Trevor Lampson (14591210),Trevor Lampson,8100,50,0
6,Daniel Hudson (14591212),Daniel Hudson,7700,50,0
7,Zander Winston (14591213),Zander Winston,7500,50,0
8,Steven Alker (14591214),Steven Alker,7300,50,0
9,Jino Sohn (14591217),Jino Sohn,6700,50,0


In [58]:
#Adjust names
df_teams.Exposure = np.where(df_teams.Name.isin(['Neil Johnson','Sam Gillis']),80,df_teams.Exposure)
df_teams

,Name + ID,Name,Salary,Exposure,Actual Exposure
0,Sahith Theegala (14591201),Sahith Theegala,10400,50,0
1,Alistair Docherty (14591202),Alistair Docherty,10200,50,0
2,Tyler Weworski (14591204),Tyler Weworski,9700,50,0
3,Nicolo Galletti (14591205),Nicolo Galletti,9400,50,0
4,Neil Johnson (14591207),Neil Johnson,9000,80,0
5,Trevor Lampson (14591210),Trevor Lampson,8100,50,0
6,Daniel Hudson (14591212),Daniel Hudson,7700,50,0
7,Zander Winston (14591213),Zander Winston,7500,50,0
8,Steven Alker (14591214),Steven Alker,7300,50,0
9,Jino Sohn (14591217),Jino Sohn,6700,50,0


In [ ]:
df_teams.Salary.mean()

In [64]:
#Now start churning
#Now set parameters for number of teams and salary limits
num_teams = 33
min_sal = 48900
max_sal = 49800

#initialize df_lineups, which will be written to csv
df_lineups = pd.DataFrame(columns=['G1','G2','G3','G4','G5','G6','Sal'])
df_lineups

#Reset df_teams
df_teams['Actual Exposure'] = 0

m = 0 #Completed teams so far
while (m < num_teams):
    #pick 6 players per probabilties
    curr_team = [p for p in lock_full_names]
    df_now = df_teams.copy()
    players = df_now['Name + ID'].tolist()
    probs = df_now['Exposure'].tolist()
    for i in range(players_left):
        p = pick_element_per_probs(players,probs)
        curr_team.append(p)
        #print(curr_team)
        #prit('picked player {}'.format(p))
        df_now = df_now[df_now['Name + ID']!=p]
        players = df_now['Name + ID'].tolist()
        probs = df_now['Exposure'].tolist()
        
    #Now curr_team is list of 6 names.  Check if salary matches
    df_now = df_teams[df_teams['Name + ID'].isin(curr_team)]
    #total_own = df_now.Ownership.sum()
    #total_own = 0 #use for small field where ownership is bonkers
    total_sal = df_now.Salary.sum() + sal_used
    #print(curr_team,total_sal)
    ##print(total_sal)
    if (total_sal < min_sal or total_sal > max_sal):
        #print('Team eliminated {}, total_sal = {}, total_own = {}'.format('team',total_sal,total_own))
        continue
        
    #Put team in order of decreasing salary
    df_now.sort_values(by='Salary',ascending=False,inplace=True)
    curr_team = lock_full_names + df_now['Name + ID'].tolist() + [total_sal]
    #print('New Team',curr_team)
    
    #Check if team has shown up before in df_lineups
    if len(df_lineups) < 1:
        old_team = False
    else:
        df3 = (df_lineups == np.array(curr_team))
        old_team = df3.transpose().all().any() #True if team is already in df_lineups
    if old_team:
        #print('Team already exists')
        continue
        
    #OK, so now we have a new team!  Add it to df_lineups
    df_lineups.loc[m] = curr_team
    #Also update 'Actual Exposure' in df_teams
    new_players = np.where(df_teams['Name + ID'].isin(curr_team),1,0)
    df_teams['Actual Exposure'] = df_teams['Actual Exposure'] + new_players
    
    #Finally increment m
    m += 1
    #print(m)
    if m%10==0:
        print('m = {}'.format(m))

    #print('Team {}, sal = {}'.format(m,total_sal))
    
df_teams[['Name + ID','Actual Exposure','Exposure','Salary']]

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


m = 10
m = 20
m = 30


,Name + ID,Actual Exposure,Exposure,Salary
0,Sahith Theegala (14591201),16,50,10400
1,Alistair Docherty (14591202),21,50,10200
2,Tyler Weworski (14591204),21,50,9700
3,Nicolo Galletti (14591205),8,50,9400
4,Neil Johnson (14591207),25,80,9000
5,Trevor Lampson (14591210),10,50,8100
6,Daniel Hudson (14591212),10,50,7700
7,Zander Winston (14591213),13,50,7500
8,Steven Alker (14591214),11,50,7300
9,Jino Sohn (14591217),7,50,6700


In [65]:
for c in df_lineups.columns:
    df_lineups.rename(columns={c:'G'},inplace=True)

In [66]:
df_lineups.head()

,G,G,G,G,G,G,G
0,Sahith Theegala (14591201),Alistair Docherty (14591202),Trevor Lampson (14591210),Daniel Hudson (14591212),Chris Figueroa (14591220),Mitchell Homb (14591221),49100
1,Sahith Theegala (14591201),Tyler Weworski (14591204),Neil Johnson (14591207),Steven Alker (14591214),Sam Gillis (14591219),Chris Figueroa (14591220),49300
2,Sahith Theegala (14591201),Tyler Weworski (14591204),Neil Johnson (14591207),Steven Alker (14591214),Blake Allen (14591218),Sam Gillis (14591219),49500
3,Alistair Docherty (14591202),Tyler Weworski (14591204),Trevor Lampson (14591210),Daniel Hudson (14591212),Jino Sohn (14591217),Blake Allen (14591218),49000
4,Alistair Docherty (14591202),Tyler Weworski (14591204),Nicolo Galletti (14591205),Daniel Hudson (14591212),Sam Gillis (14591219),Mitchell Homb (14591221),49800


In [67]:
#If you're good with actual exposure, then write it!
df_lineups.to_csv('Lineups/DK Entires.csv',index=False)

In [ ]:
df_lineups['All'] = df_lineups.sum(axis=1)
df_lineups.head(2)

In [ ]:
df_lineups[(df_lineups.All.str.contains('Gaugert')) &
          (df_lineups.All.str.contains('Dalen'))]